# Cross-validated raster plot

Find depth neurons based on subset of the trials and plot raster from the other subset.

## Finding depth neuron

Start with an example

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import scipy
from matplotlib import pyplot
import flexiznam as flz
from cottage_analysis.depth_analysis.depth_preprocess import process_params
from scipy.optimize import curve_fit
from cottage_analysis.depth_analysis.plotting.plotting_utils import calculate_R_squared, get_PSTH
from v1_depth_analysis.cosyne_poster_2023 import raster_crossvalidate as rac

In [ ]:
# Define sessions
project = "hey2_3d-vision_foodres_20220101"
V1_sessions = {
    "PZAH6.4b": [
        "S20220512",
        "S20220511",
        "S20220429",
        "S20220419",
        "S20220426",
        "S20220506",
        "S20220503",
        "S20220505",
        "S20220516",
    ],
    "PZAG3.4f": [
        "S20220504",
        "S20220426",
        "S20220421",
        "S20220419",
        "S20220505",
        "S20220503",
        "S20220510",
        "S20220512",
        "S20220511",
    ],
}

VERSION =1 
target_folder = Path(flz.PARAMETERS['data_root']['processed']).parent / f'presentations/Cosyne2023/ver{VERSION}/raster_summary_crossvalidated/'
if not target_folder.is_dir():
    target_folder.mkdir(exist_ok=True)


speed_thr_cal = 0.2  
MIN_SIGMA=0.5
# initialise random generator
seed = 796
rng = np.random.default_rng(seed=seed)
# which proportion of the trials are used for the fit vs raster
prop_fit = 0.5

In [ ]:
flm_sess = flz.get_flexilims_session(project_id=project)

## Iterate on sessions

In [ ]:
# select the session
REDO = False

all_psths = dict()
all_fits = dict()
all_anovas = dict()
for mouse, sessions in V1_sessions.items():
    for session in sessions:
        print(f"Doing session {session} of {mouse}")
        # get the data
        sess_entity = flz.get_entity(
            name=f"{mouse}_{session}", datatype="session", flexilims_session=flm_sess
        )

        recordings = flz.get_children(
            parent_id=sess_entity.id,
            children_datatype="recording",
            flexilims_session=flm_sess,
        )
        recording = recordings[recordings.protocol == "SpheresPermTubeReward"]
        assert len(recording) == 1
        recording = recording.iloc[0]

        fnames = [
            f"{'_'.join(recording.genealogy)}_psth_{seed}_{prop_fit}.npy",
            f"{'_'.join(recording.genealogy)}_gaussian_fit_{seed}_{prop_fit}.csv",
            f"{'_'.join(recording.genealogy)}_anovas_{seed}_{prop_fit}.npy",
        ]

        if not REDO and all([(target_folder / f).exists() for f in fnames]):
            print("already analysed, reload", flush=True)
            data = None
            stim_dict_fit = None  # this will make the analysis crash if it tries to run
            stim_dict_raster = None
        else:

            data = rac.get_recording_data(recording, flm_sess, two_photon=True)
            stim_dict_fit, stim_dict_raster = rac.cut_stim_dict(
                stim_dict=data["stim_dict"], prop_fit=prop_fit, rng=rng
            )

        all_fits[f"{mouse}_{session}"] = rac.get_or_load_fit(
            recording,
            seed,
            prop_fit,
            stim_dict_fit,
            target_folder,
            speed_thr_cal,
            MIN_SIGMA,
            data=data,
            redo=REDO,
        )
        all_anovas[f"{mouse}_{session}"] = rac.get_or_load_anova(
            recording,
            target_folder,
            seed,
            prop_fit,
            data=data,
            stim_dict=stim_dict_fit,
            speed_thr_cal=speed_thr_cal,
            redo=REDO,
        )

        all_psths[f"{mouse}_{session}"] = rac.get_or_load_psth(
            recording,
            target_folder,
            seed,
            prop_fit,
            data=data,
            stim_dict_raster=stim_dict_raster,
            redo=REDO,
        )


# Do the raster

In [ ]:
if data is None:
    # load a random rec to read depth
    data = rac.get_recording_data(recording, flm_sess, two_photon=True)
depth_list = data['param_logger'].Depth.unique()
depth_list = np.round(depth_list, 2)
depth_list = depth_list[~np.isnan(depth_list)].tolist()
depth_list.remove(-99.99)
depth_list.sort()
depth_list[-1] = 6  # dirty mix up between 6.0 and 6


In [ ]:
# Plot it
sessions = list(all_psths.keys())
all_neurons_psth = np.vstack([all_psths[k] for k in sessions])
all_neurons_anova = np.hstack([all_anovas[k] for k in sessions])
fits = [all_fits[k] for k in sessions]
all_neurons_preferred_depths = np.hstack(
    [np.array(np.exp(f.x0_logged.values.astype("float32"))) for f in fits]
)
all_neurons_rsquare = np.hstack(
    [np.array(f.r_sq.values.astype("float32")) for f in fits]
)


In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(all_neurons_rsquare, bins=np.arange(0, 0.2, 0.001))

In [ ]:
# Colormap
from matplotlib.colors import ListedColormap
N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1, 1, N)
vals[:, 1] = np.linspace(1, 0, N)
vals[:, 2] = np.linspace(1, 0, N)
WhRdcmap = ListedColormap(vals)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4,12))
plt.subplot(3,1,1)
plt.title("All neurons")
depth_list = []
valid = np.ones(len(all_neurons_preferred_depths), dtype=bool)
order = all_neurons_preferred_depths.argsort()
psth = all_neurons_psth[order[valid]]
# normalise each neuron
neuron_max = np.nanmax(psth, axis=1)[:, np.newaxis]
neuron_min = np.nanmin(psth, axis=1)[:, np.newaxis]
normed_psth = (psth - neuron_min) / (neuron_max-neuron_min)
plt.imshow(normed_psth, vmax=1, aspect='auto', cmap=WhRdcmap)

plt.subplot(3,1,2)
plt.title("Anova <0.05")
valid = all_neurons_anova < 0.05
psth = all_neurons_psth[order[valid]]
# normalise each neuron
neuron_max = np.nanmax(psth, axis=1)[:, np.newaxis]
neuron_min = np.nanmin(psth, axis=1)[:, np.newaxis]
normed_psth = (psth - neuron_min) / (neuron_max-neuron_min)

plt.imshow(normed_psth, vmax=1, aspect='auto', cmap=WhRdcmap)

plt.subplot(3,1,3)
fit_threshold = 0.025
plt.title(f"Fit r > {fit_threshold}")
valid = all_neurons_rsquare > fit_threshold
psth = all_neurons_psth[order[valid]]
# normalise each neuron
neuron_max = np.nanmax(psth, axis=1)[:, np.newaxis]
neuron_min = np.nanmin(psth, axis=1)[:, np.newaxis]
normed_psth = (psth - neuron_min) / (neuron_max-neuron_min)
plt.imshow(normed_psth, vmax=1, aspect='auto', cmap=WhRdcmap, origin='lower')
for ax in plt.gcf().axes:
    ax.set_xticks(np.arange(normed_psth.shape[1])[30::60],
                  (np.array(depth_list)*100).astype('int'), fontsize=15,
                   rotation = 30)
    for x in np.arange(normed_psth.shape[1])[61::60]:
        ax.axvline(x, color='k', lw=1)